In [1]:
import os
import pandas as pd
import json
import shutil

In [2]:
# warning: this Jupyter notebook uses only Windows-style paths.
a_dir = 'output/mdpi/' # change to plos/elife/mdpi to see others
r_dir = a_dir + "reviewed/"


In [22]:
arts = pd.DataFrame({'doi': os.listdir(a_dir)})


In [3]:
rarts = pd.DataFrame({'doi': os.listdir(r_dir)})

In [24]:
len(arts), len(rarts)

(607745, 37756)

In [4]:
rarts = rarts[rarts.doi.map(lambda x: os.path.isdir(r_dir + x))]


In [25]:
arts = arts.loc[arts.doi.map(lambda x: os.path.isdir(a_dir + x))]
len(arts), len(rarts)

(607743, 37755)

In [5]:
rarts['path'] = r_dir + rarts.doi


In [26]:
arts['path'] = a_dir + arts.doi
arts = arts[arts.path.map(lambda x: not x.endswith("reviewed"))]

In [27]:
def getj(a):
    path = os.path.join(a, "metadata.json")
    if not os.path.exists(path):
        print("no meta for a =", a)
        return pd.NA
    fp = open(path, encoding='utf-8')
    j = json.load(fp)
    fp.close()
    return j


In [28]:
arts['meta'] = arts.path.map(getj)
rarts['meta'] = rarts.path.map(getj)

In [29]:
arts['has_r'] = arts.meta.map(lambda a: a['has_reviews'])
sum(arts.has_r), "<- unfortunate..."

(114062, '<- unfortunate...')

In [30]:
# arts['has_suba'] = arts.path.map(lambda p: os.path.exists(p+"/sub-articles"))

In [31]:
# sum(arts.has_suba), len(arts)

In [8]:
rarts['has_suba'] = rarts.path.map(lambda p: os.path.exists(p+"/sub-articles"))

In [10]:
sum(rarts.has_suba), len(rarts)

(90066, 151817)

In [34]:
import shutil

##

In [35]:
r_count = 0
has_r_count = 0
for rart in os.listdir(r_dir):
    if os.path.isdir(r_dir+rart):
        fp = open(r_dir+rart+"/metadata.json", 'r', encoding='utf-8')
        j = json.load(fp)
        fp.close()
        if j['has_reviews']: 
            has_r_count += 1
            if os.path.exists(r_dir+rart+"/sub-articles"):
                if len(os.listdir(r_dir+rart+"/sub-articles")) == 0:
                    print("Removing empty sub-a folder for",rart)
                    shutil.rmtree(r_dir+rart+"/sub-articles")
                else:
                    r_count+=1
        else:
            print(rart,"doesnt actually have reviews?! Deleting this dir...") 
            shutil.rmtree(r_dir+rart)

In [36]:
print(r_count, "out of", len(os.listdir(r_dir)))

33976 out of 37756


In [37]:
has_r_count

37755

##

In [47]:
reviewed_arts = arts[arts['meta'].map(lambda x: x['has_reviews'])] 

In [61]:
reviewed_arts['new_path'] = reviewed_arts.path.apply(lambda x: '/'.join(x.split('/')[0:2]) + '/reviewed/' + x.split('/')[2])

C:\Users\acted\AppData\Local\Temp\ipykernel_17440\2441550324.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviewed_arts['new_path'] = reviewed_arts.path.apply(lambda x: '/'.join(x.split('/')[0:2]) + '/reviewed/' + x.split('/')[2])


In [68]:
reviewed_arts['outasd'] = reviewed_arts.apply(lambda x: shutil.copytree(x.path, x.new_path), axis=1)

C:\Users\acted\AppData\Local\Temp\ipykernel_17440\3512361509.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviewed_arts['outasd'] = reviewed_arts.apply(lambda x: shutil.copytree(x.path, x.new_path), axis=1)
